In [1]:
from BCBio import GFF
path = r"F:\01 科研项目数据-2022\06-基础AI\human.gtf"
from BCBio import GFF
from collections import Counter


In [2]:
biotype = []

in_handle = open(path,'r')

for rec in GFF.parse(in_handle):
    for feature in rec.features:
        if feature.type == "gene":
            biotype.append(feature.qualifiers['gene_biotype'][0])
            
in_handle.close()
len(biotype)
biotype[0:15]

Counter(biotype)

Counter({'transcribed_unprocessed_pseudogene': 925,
         'unprocessed_pseudogene': 2627,
         'miRNA': 1879,
         'lncRNA': 16887,
         'protein_coding': 19941,
         'processed_pseudogene': 10168,
         'snRNA': 1901,
         'transcribed_processed_pseudogene': 498,
         'misc_RNA': 2211,
         'TEC': 1061,
         'transcribed_unitary_pseudogene': 137,
         'snoRNA': 943,
         'scaRNA': 49,
         'rRNA_pseudogene': 500,
         'unitary_pseudogene': 97,
         'polymorphic_pseudogene': 42,
         'pseudogene': 18,
         'rRNA': 52,
         'IG_V_pseudogene': 188,
         'ribozyme': 8,
         'sRNA': 5,
         'TR_V_gene': 106,
         'TR_V_pseudogene': 33,
         'TR_D_gene': 4,
         'TR_J_gene': 79,
         'TR_C_gene': 6,
         'TR_J_pseudogene': 4,
         'IG_C_gene': 14,
         'IG_C_pseudogene': 9,
         'IG_J_gene': 18,
         'IG_J_pseudogene': 3,
         'IG_D_gene': 37,
         'IG_V_gene': 144,


In [3]:
def get_gene_info(gencode_filename):
    gene_dict = {}
    with open(gencode_filename, "r") as gencode_file, open("gene_dict.csv", "w+") as gene_dict_file:
        effective_genes_num = 0

        for line in gencode_file:  # read line by line

            if line[0:3] != "chr":  # skip useless lines
                continue

            # separate line into 9 parts
            # …… \t …… \t line_info_type \t …… \t …… \t …… \t …… \t …… \t gene_info \t
            line_info = line.split("\t")
            if line_info[2] != "gene":  # skip unimportant lines
                continue

            # eg.gene_id "ENSG00000223972.5"; gene_type "transcribed_unprocessed_pseudogene"; gene_name "DDX11L1"; ……
            gene_info = line_info[8].split(" ")
            gene_id = gene_info[1].split(".")[0][1:]
            gene_name = gene_info[5][1:-2]

            gene_dict[gene_id] = gene_name
            gene_dict_info = gene_id + "," + gene_name + "\n"
            gene_dict_file.write(gene_dict_info)

            effective_genes_num += 1

        print("Effective genes number found in " + gencode_filename + " is " + str(effective_genes_num))
        return gene_dict


def transformENSGid(gencode_filename, tcga_filename):
    gene_dict = get_gene_info(gencode_filename)

    transformedENSGid_num = 0

    with open(tcga_filename, "r") as tcga_data, open("TCGAfpkm_transformed.xls", "w+") as new_tcga_data_file:
        for line in tcga_data:

            if line[0:4] != "ENSG":  # skip useless lines
                new_tcga_data_file.write(line)
                continue

            # separate line into several parts
            line_info = line.split("\t")

            # get gene_id
            gene_id = line_info[0].split(".")[0]

            # transform
            try:
                line_info[0] = gene_dict[gene_id]
            except:
                continue  # skip useless lines

            # generate new line
            new_line = "\t"
            new_line = new_line.join(line_info)
            new_tcga_data_file.write(new_line)

            transformedENSGid_num += 1

        print("Transformed ENSGID in " + tcga_filename + " is " + str(transformedENSGid_num) + ".")


In [4]:
gene_dict = get_gene_info(r"F:\01 科研项目数据-2022\06-基础AI\human.gtf")

Effective genes number found in F:\01 科研项目数据-2022\06-基础AI\human.gtf is 0


In [5]:
gencode_filename = r"F:\01 科研项目数据-2022\06-基础AI\human.gtf"

In [6]:
gene_dict = {}
with open(gencode_filename, "r") as gencode_file:
    effective_genes_num = 0
    for line in gencode_file: 
        if line[0:2] == "#!":
            continue
#         print(line)
        line_info = line.split("\t")
        
        if line_info[2] != "gene":  # skip unimportant lines
            continue
        gene_info = line_info[8].split(" ")
        gene_id = gene_info[1]
        gene_name = gene_info[5]

        gene_id=gene_id.replace('"','')
        new_gene_id=gene_id.replace(';','')
        gene_name=gene_name.replace('"','')
        new_gene_name=gene_name.replace(';','')
        gene_dict[new_gene_id] = new_gene_name
        effective_genes_num += 1

In [7]:
len(gene_dict)

60624

In [10]:
ids = []
namses = []

for key, val in gene_dict.items():
    ids.append(key)
    namses.append(val)

In [11]:
import pandas as pd

In [12]:
gene_names = pd.DataFrame(data = {"gene_id" : ids, "genn_name": namses})
gene_names.to_csv(r"F:\01 科研项目数据-2022\06-基础AI\gene_names.csv")